In [1]:
import pymongo
import plotly.express as px
import pandas as pd
import numpy as np

In [2]:
conn_str = f"mongodb://localhost"
mongo_client = pymongo.MongoClient(conn_str, port=27018)

In [ ]:
# Old code to merge separate databases' content
# statistics_df = pd.DataFrame(mongo_client.test.features.find({}))
# statistics_df['dzn'] = statistics_df['dzn'].fillna(value="")
# statistics_df["id"] = statistics_df["mzn"] + "/" + statistics_df["dzn"]
# statistics_df = statistics_df[['id' ,'statistics']]


# labels_df = pd.DataFrame(mongo_client.test.labels.find({}))
# labels_df['dzn'] = labels_df['dzn'].fillna(value="")
# labels_df["id"] = labels_df["mzn"] + "/" + labels_df["dzn"]
# del labels_df["statistics"]


# # Clean up data
# all_data = pd.merge(statistics_df, labels_df, on="id")

In [11]:
all_data = pd.DataFrame(mongo_client.done_soon.problems.find({}))

0         True
1         True
2         True
3         True
4         True
         ...  
17106    False
17107     True
17108    False
17109    False
17110     True
Name: solved, Length: 17111, dtype: bool

In [6]:
removed_exceptions = all_data[(all_data.statistics.str.len() == 200) | (all_data.solved == True)]
timeout_after_hour = removed_exceptions[removed_exceptions.statistics.str.len() > 100]
cancels_before_6_minutes = removed_exceptions[removed_exceptions.statistics.str.len() <= 10]
running_after_6_minutes = removed_exceptions[removed_exceptions.statistics.str.len() > 10]
# all_data = all_data[all_data.solved == False]

print(len(timeout_after_hour))
print(len(running_after_6_minutes))
print(len(cancels_before_6_minutes))




93
857
14059


In [13]:
all_data[all_data.solved == False].time_to_solution

7       NaN
8       NaN
10      NaN
19      NaN
20      NaN
         ..
17104   NaN
17105   NaN
17106   NaN
17108   NaN
17109   NaN
Name: time_to_solution, Length: 2237, dtype: float64

In [10]:
px.histogram(removed_exceptions[removed_exceptions.solved == True], x="time_to_solution", nbins=20)

In [80]:
np.all(np.array(removed_exceptions[removed_exceptions.solved == False].statistics.map(len)) == 200)
len(removed_exceptions[removed_exceptions.time_to_solution >= 7200000].index)
len(removed_exceptions[removed_exceptions.solved == False].index)
removed_exceptions.type.value_counts()
# len(removed_exceptions[removed_exceptions.time_to_solution <3600000].index)
# removed_exceptions[removed_exceptions.solved == False].statistics.map(len).value_counts()[200]

OPT    13058
SAT      738
Name: type, dtype: int64

In [8]:
px.histogram(removed_exceptions[removed_exceptions.time_to_solution > 1000], x="time_to_solution", nbins=20)

In [24]:
all_data['percentage'] = all_data['statistics'].map(lambda x: len(x) / 2)
fig = px.histogram(all_data, x="percentage", nbins=40, labels={"percentage": "Percentage of TL"})
fig.add_vline(5)
fig.show()

In [27]:
running_after_6_minutes['percentage'] = running_after_6_minutes['statistics'].map(lambda x: len(x) / 2)
fig = px.histogram(running_after_6_minutes, x="percentage", nbins=40, labels={"percentage": "Percentage of TL"})
fig.add_vline(50)
fig.show()

/tmp/ipykernel_57680/2481753301.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
def point_to_percentile_indexes(p):
    return [(round(s['percent']*100000, 2) + 0.5) / 2 for s in p['statistics']]


In [68]:
# def plot_feature_over_time(p, feature):
#     percentages = point_to_percentile_indexes(p)
#     df = pd.DataFrame(zip(percentages, [s['features'][feature] for s in p['statistics']]), columns=['percent', feature])
#     fig = px.line(df, x="percent", y=feature, title=f"{feature} per percent")
#     fig.update_xaxes(range=[0, 100])
#     fig.show()

# index = 1
# plot_feature_over_time(all_data.loc[index], "decision_level_engine")
# plot_feature_over_time(all_data.loc[index], "ewma_decision_level_engine")

In [69]:
#Calculate additonal features
time_out = 7200 #time in s (or 2hrs)
time_diff = 0.05*time_out

#combined features
for df in df_list:
    df["unassnVar"]   = (2^df.vars) - df.nodes
    df["fracFailUnassn"]     = df.failures / df.unassnVar         # num failures/ num open nodes
    df["fracOpenVisit"]  = (df.vars - df.nodes) / df.nodes       # ratio of open nodes to visited nodes (how much of soln space explored)
    df["fracBoolVars"]     = df.boolVars / df.vars                 # num bools / total num of vars
    df["fracPropVars"]     = df.propagations / df.vars        # num propagations/ total num of vars
    df["frac_unassigned"] = df.unassnVar / df.vars  # current assignments/ total vars
    df["fracLongClauses"] = df.long_vars / (df.long_vars + df.bin + df.tern)         # fraction of learnt clauses that have more than 3 literals
    df["freqBackjumps"]  = df.back_jumps/df.search_time
    df.fillna(0, inplace =True)  


#global rate of change features
G_roc_best_objective = [ (df_10.best_objective - df_5.best_objective)/time_diff,
                           (df_15.best_objective - df_10.best_objective)/time_diff,                 
                           (df_20.best_objective - df_15.best_objective)/time_diff]

G_roc_nodes = [ (df_10.nodes - df_5.nodes)/time_diff,
                    (df_15.nodes - df_10.nodes)/time_diff,                 
                    (df_20.nodes - df_15.nodes)/time_diff] 

G_roc_failures = [ (df_10.failures - df_5.failures)/time_diff,
                    (df_15.failures - df_10.failures)/time_diff,                 
                    (df_20.failures - df_15.failures)/time_diff] 

G_roc_backjumps = [ (df_10.back_jumps - df_5.back_jumps)/time_diff,
                    (df_15.back_jumps - df_10.back_jumps)/time_diff,                 
                    (df_20.back_jumps - df_15.back_jumps)/time_diff]

G_roc_propagations = [ (df_10.propagations- df_5.propagations)/time_diff,
                    (df_15.propagations - df_10.propagations)/time_diff,                 
                    (df_20.propagations - df_15.propagations)/time_diff]  

G_roc_solutions = [ (df_10.solutions - df_5.solutions)/time_diff,
                    (df_15.solutions - df_10.solutions)/time_diff,                 
                    (df_20.solutions - df_15.solutions)/time_diff]   
'''
G_roc_restarts = [ (df_10.restarts - df_5.restarts)/time_diff,
                    (df_15.restarts - df_10.restarts)/time_diff,                 
                    (df_20.restarts - df_15.restarts)/time_diff]                                                                              
'''


# add glbal rate of change features to dfs
count = 0
for k in [df_10, df_15, df_20]:
  k["objectiveGROC"] = G_roc_best_objective[count]
  k["nodesGROC"] = G_roc_nodes[count]
  k["failureGROC"] = G_roc_failures[count]
  k["backjumpsGROC"] = G_roc_backjumps[count]
  k["propagationsGROC"] = G_roc_propagations[count]
  k["solutionsGROC"] = G_roc_solutions[count]
  count = count + 1

NameError: name 'df_list' is not defined